In [15]:
#Import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [16]:
#Load data from database
engine = create_engine('sqlite:///data/02_stg//stg_disaster_response.db')

# Create a dataframe from the engine
df = pd.read_sql_table('stg_disaster_response', engine)

#feature columns
feature_columns =['message']

#target columns
target_columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
                   'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 
                   'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 
                   'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
                   'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

# Define the features and target variables X and Y
X = df[feature_columns].values
Y = df[target_columns].values

In [17]:
# Constants
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_place_holder = "urlplaceholder"

In [18]:
def detect_and_replace_urls(text):
    """
    Detects URLs in the text and replaces them with a placeholder.
    
    :param text: String containing the original text.
    :return: Text with URLs replaced by placeholder.
    """
    return re.sub(url_regex, url_place_holder, text)

def tokenize_text(text):
    """
    Tokenizes the text into individual words/tokens.
    
    :param text: String to tokenize.
    :return: List of tokens.
    """
    return word_tokenize(text)

def clean_tokens_generator(tokens):
    """
    Cleans and normalizes tokens using lemmatization. Uses a generator to yield tokens one at a time.
    
    :param tokens: Iterable of tokens to clean.
    :yield: Cleaned token one at a time.
    """
    lemmatizer = WordNetLemmatizer()
    for token in tokens:
        yield lemmatizer.lemmatize(token.lower().strip())

def preprocess_text(text):
    """
    Preprocesses the text by detecting URLs, tokenizing, and cleaning the tokens using a generator.
    
    :param text: String containing the original text.
    :return: List of preprocessed and cleaned tokens.
    """
    text_with_placeholder = detect_and_replace_urls(text)
    tokens = tokenize_text(text_with_placeholder)
    clean_tokens_gen = clean_tokens_generator(tokens)
    # Convert the generator to a list if you need to work with all tokens at once
    cleaned_tokens_list = list(clean_tokens_gen)
    return cleaned_tokens_list

In [20]:
results = preprocess_text(X[0])
print(results)

#Something is off with the URL functionality. Fix this before moving on

TypeError: cannot use a string pattern on a bytes-like object